In [14]:
import json

def get_llmbar_res():
    with open('avg_results.json', 'r') as f:
        data = json.load(f)
    max_num = max(data.keys())
    return data[max_num], max_num

def get_other_res():
    with open('result.json', 'r') as f:
        data = json.load(f)

    other_data =  [x for x in data if 'LLMBar' not in x]
    for x in other_data:
        testset_name = None
        for name in ['PandaLM', 'auto-j', 'MT-Bench']:
            if name in x:
                testset_name = name
                break
        x["testset"] = testset_name
        x['metric'] = x[testset_name]
        del x[testset_name]
    return other_data


In [15]:
# llmbar_res = get_llmbar_res()
other_res = get_other_res()

In [25]:
llmbar_res, max_num = get_llmbar_res()

In [5]:
x=[{
        "PandaLM": {
            "Accuracy": 0.6646666666666665,
            "Precision": 0.6471062227343476,
            "Recall": 0.665331998665332,
            "F1": 0.6560925603971778
        },
        "prompt_type": "prompt1"
    },
    {
        "PandaLM": {
            "Accuracy": 0.6846846846846847,
            "Precision": 0.6638175381503862,
            "Recall": 0.6846846846846847,
            "F1": 0.6740896590309133
        },
        "prompt_type": "prompt1"
    }]

y={
        "PandaLM": {
            "Accuracy": 0.6646666666666665,
            "Precision": 0.6471062227343476,
            "Recall": 0.665331998665332,
            "F1": 0.6560925603971778
        },
        "prompt_type": "prompt1"
    }
for metric in y["PandaLM"].keys():
    y["PandaLM"][metric] = (x[0]["PandaLM"][metric]+ x[1]["PandaLM"][metric])/2

In [8]:
str(y).replace('\'', '"')

'{"PandaLM": {"Accuracy": 0.6746756756756755, "Precision": 0.6554618804423669, "Recall": 0.6750083416750083, "F1": 0.6650911097140455}, "prompt_type": "prompt1"}'

In [10]:
list(set([x['prompt_type'] for x in other_res]))

['prompt_CoT',
 'prompt_vanila_rule',
 'prompt1',
 'prompt_vanila',
 'prompt1_cn',
 'prompt_CoT_rule']

In [32]:
def convert_to_html_table(data_list):
    # 定义表格的列名和行名
    column_names = ['PandaLM', 'auto-j', 'MT-Bench']
    row_names = ['prompt1', 'prompt1_cn', 'prompt_vanila', 'prompt_CoT', 'prompt_vanila_rule', 'prompt_CoT_rule']

    # 创建一个字典来存储数据
    data_dict = {col: {row: {'Accuracy': '-', 'F1': '-'} for row in row_names} for col in column_names}

    # 遍历列表中的每一条数据,并将数据填充到字典中
    for item in data_list:
        testset = item['testset']
        prompt_type = item['prompt_type']
        accuracy = item['metric']['Accuracy']
        f1 = item['metric']['F1']

        if testset in column_names and prompt_type in row_names:
            data_dict[testset][prompt_type]['Accuracy'] = round(accuracy * 100, 1)
            data_dict[testset][prompt_type]['F1'] = round(f1 * 100, 1)

    # 构建 HTML 表格
    table_html = '<table>\n'
    table_html += '  <tr>\n'
    table_html += '    <th colspan="2"></th>\n'
    for col in column_names:
        table_html += f'    <th colspan="2" style="text-align: center;">{col}</th>\n'
    table_html += '  </tr>\n'
    table_html += '  <tr>\n'
    table_html += '    <th colspan="2">Prompt Type</th>\n'
    for _ in column_names:
        table_html += '    <th style="text-align: center;">Accuracy</th>\n'
        table_html += '    <th style="text-align: center;">F1</th>\n'
    table_html += '  </tr>\n'

    for row in row_names:
        table_html += '  <tr>\n'
        table_html += f'    <th colspan="2">{row}</th>\n'
        for col in column_names:
            accuracy = data_dict[col][row]['Accuracy']
            f1 = data_dict[col][row]['F1']
            table_html += f'    <td style="text-align: center;">{accuracy}</td>\n'
            table_html += f'    <td style="text-align: center;">{f1}</td>\n'
        table_html += '  </tr>\n'

    table_html += '</table>'

    return table_html

In [33]:
with open('table.html', 'w') as f:
    f.write(convert_to_html_table(other_res))

In [30]:
def generate_html_table(data_list):
    # 定义表格的列名和行名
    column_names = ['Natural', 'Neighbor', 'GPTInst', 'GPTOut', 'Manual', 'Average', 'Average_sum']
    row_names = ['prompt1', 'prompt1_cn', 'prompt_vanila', 'prompt_CoT', 'prompt_vanila_rule', 'prompt_CoT_rule']

    # 创建一个字典来存储数据
    data_dict = {row: {col: '-' for col in column_names} for row in row_names}

    # 遍历列表中的每一条数据,并将数据填充到字典中
    for item in data_list:
        prompt_type = item['prompt_type']
        for col in column_names[:5]:
            if col in item['LLMBar']:
                data_dict[prompt_type][col] = round(item['LLMBar'][col]['Accuracy'] * 100, 1)

        # 计算第6列的Average
        cols_2_5 = [data_dict[prompt_type][col] for col in column_names[1:5]]
        cols_2_5 = [val for val in cols_2_5 if isinstance(val, (int, float))]
        if cols_2_5:
            data_dict[prompt_type][column_names[5]] = round(sum(cols_2_5) / len(cols_2_5), 1)

        # 计算第7列的Average
        cols_1_5 = [data_dict[prompt_type][col] for col in column_names[:5]]
        cols_1_5 = [val for val in cols_1_5 if isinstance(val, (int, float))]
        if cols_1_5:
            data_dict[prompt_type][column_names[6]] = round(sum(cols_1_5) / len(cols_1_5), 1)

    # 构建 HTML 表格
    table_html = '<table>\n'
    table_html += '  <tr>\n'
    table_html += '    <th rowspan="2">Prompt Type</th>\n'
    table_html += '    <th rowspan="2">Natural</th>\n'
    table_html += '    <th colspan="5">ADVERSARIAL</th>\n'
    table_html += '    <th rowspan="2">Average</th>\n'
    table_html += '  </tr>\n'
    table_html += '  <tr>\n'
    for col in column_names[1:6]:
        table_html += f'    <th>{col}</th>\n'
    table_html += '  </tr>\n'

    for row in row_names:
        table_html += '  <tr>\n'
        table_html += f'    <td><b>{row}</b></td>\n'
        for col in column_names:
            if col != 'Natural':
                table_html += f'    <td style="text-align: center;">{data_dict[row][col]}</td>\n'
            else:
                table_html += f'    <td>{data_dict[row][col]}</td>\n'
        table_html += '  </tr>\n'

    table_html += '</table>'

    return table_html

In [31]:
with open('table.html', 'w') as f:
    f.write(generate_html_table(llmbar_res))